# Подготовка среды

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# os.chdir('/content')

In [32]:
import shutil
shutil.copytree("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/embeddings/bert-base-multilingual-cased-polit3/bin", "/content/data/bin2")

'/content/data/bin2'

In [33]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [34]:
folder = "bert-base-multilingual-cased-polit3"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

In [35]:
!pip install transformers
!pip install razdel

In [36]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [37]:
# def my_dump(file, name):
#     with open("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/" + name + ".pic", 'wb') as f:
#         pc.dump(file, f)

# def my_load(name):
#     with open("./" + name + ".pic", 'rb') as f:
#         return pc.load(f)

In [38]:
def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [ ]:
%%time
name = "bin/all_cyberleninka_polit2"
# name = "all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 7.97 s, sys: 2.41 s, total: 10.4 s
Wall time: 15 s


In [ ]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

2348859

##process

In [ ]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings2(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)

    token_vecs_sum = list(map(sum_tensors, token_embeddings))

    new_tokens = []
    new_token_vecs = []
    j = -1
    for i in range(len(tokenized_text)):
        if len(tokenized_text[i]) > 2 and tokenized_text[i][0] == "#" and tokenized_text[i][1] == "#":
            new_tokens[j] = new_tokens[j] + tokenized_text[i][2:]
            new_token_vecs[j].append(token_vecs_sum[i])
        else:
            new_tokens.append(tokenized_text[i])
            new_token_vecs.append([token_vecs_sum[i]])
            j+=1
    new_token_vecs2 = []
    for i in new_token_vecs:
        new_token_vecs2.append(sum(i)/len(i))
    return new_tokens, new_token_vecs2

In [ ]:
# %%time
# batch = 1000
# start_time = time.time()

# ftom_b = 707
# to_b = 1000

# print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

# for j in range(ftom_b, to_b):
#     pr_tr = {}
#     temp_time = time.time()
#     for i in range(j * batch, (j + 1) * batch):
#         tt, tvs = get_embeddings2(sent[i])
#         for i in range(len(tt)):
#             if tt[i].isalpha() and len(tt[i]) > 2 :
#                 if tt[i] in pr_tr:
#                     pr_tr[tt[i]].append(tvs[i])
#                 else:
#                     pr_tr[tt[i]] = [tvs[i]]

#     with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
#         pc.dump(pr_tr, f)

#     t = time.time()
#     print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# # for i in pr_tr:
# #     print(i, len(pr_tr[i]))

In [ ]:
%%time
batch = 1000
start_time = time.time()

ftom_b = 477
to_b = 350
step = -1

print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

for j in range(ftom_b, to_b, step):
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings2(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha() and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

senteces from 477000 to 350000
done 00477 --- time: 322.00382566452026	--- total: 322.0044560432434
done 00476 --- time: 319.50530767440796	--- total: 641.5230407714844
done 00475 --- time: 302.2564516067505	--- total: 943.79460978508
done 00474 --- time: 314.1696629524231	--- total: 1257.9801771640778
done 00473 --- time: 292.3994755744934	--- total: 1550.3905000686646
done 00472 --- time: 318.21873354911804	--- total: 1868.620866060257
done 00471 --- time: 306.10651564598083	--- total: 2174.741882801056
done 00470 --- time: 350.25730443000793	--- total: 2525.012624979019
done 00469 --- time: 303.7279849052429	--- total: 2828.7616765499115
done 00468 --- time: 322.67252111434937	--- total: 3151.4465956687927
done 00467 --- time: 307.3247399330139	--- total: 3458.7836270332336
done 00466 --- time: 291.5112986564636	--- total: 3750.3068749904633
done 00465 --- time: 318.64526176452637	--- total: 4068.963541984558
done 00464 --- time: 322.36925864219666	--- total: 4391.347265958786
done 

OSError: ignored

#aggregation of embeddings M



In [ ]:
%%time
count_t = {}
sem_emb = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in count_t:
            count_t[i] = 0
            sem_emb[i] = data[i]
        else:
            count_t[i] += 1
            sem_emb[i] = sem_emb[i] + data[i]

res = {}
for key in count_t:
    res[key] = sem_emb[key] / count_t[key]
    # print(name)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

#aggregation of embeddings



In [ ]:
# folder = "bert-base-multilingual-cased-polit"

##list of files

In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [ ]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    # with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
    with open("/content/data/bin2/{}".format(name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 10min 15s, sys: 27.6 s, total: 10min 42s
Wall time: 11min 27s


In [ ]:
my_dump(from_files, "./embeddings/{}/from_files".format(folder))
my_dump(all_count, "./embeddings/{}/all_count".format(folder))

## aggregate

In [39]:
from_files = my_load("./embeddings/{}/from_files".format(folder))
all_count = my_load("./embeddings/{}/all_count".format(folder))

In [40]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

[('в', 406148),
 ('и', 347208),
 ('с', 159911),
 ('на', 112761),
 ('не', 98073),
 ('что', 71344),
 ('к', 66518),
 ('а', 66409),
 ('как', 63271),
 ('по', 50981),
 ('the', 50276),
 ('о', 47417),
 ('м', 43266),
 ('of', 38391),
 ('для', 37934),
 ('это', 36738),
 ('из', 36447),
 ('его', 35772),
 ('г', 33240),
 ('но', 31132)]

In [41]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(88, 1602, 8764, 467405)

In [42]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

62499

In [43]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        # with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        with open("/content/data/bin2/{}".format(name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [44]:
# batch = 500
# st_time = time.time()

# ftom_b = 0
# to_b = 2
# step = 1

# print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

# all_emb = {}
# for b in range(ftom_b, to_b, step):
#     t_time = time.time()
#     words = all_words[b*batch:(b+1)*batch]
#     emb = get_all_embeddings(words)
#     res_emb = sum_embeddings(emb)
    
#     with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
#         pc.dump(res_emb, f)

#     print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

In [45]:
def get_batch_number(max_i, from_i = 0):
    
    files_r = os.listdir("./embeddings/{}/ready".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1]))
    diff = set(range(from_i, max_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(b))
    my_dump("file", "./embeddings/{}/ready/{}_{:05d}_empty".format(folder,batch, b))

batch = 1000
from_i = 4

st_time = time.time()
print(" max batchs = {}".format( len(all_words)//batch))

all_emb = {}
l = get_batch_number(len(all_words)//batch + 1, from_i)
while len(l) > 0:
    b = l[0]
    reserve_batch(b)
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))
    l = get_batch_number(len(all_words)//batch + 1, from_i)

 max batchs = 62
reserve 6


FileNotFoundError: ignored

## delete trash

In [17]:
files = os.listdir("./embeddings/{}/ready".format(folder))
trash_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        if len(i.split('.')[0].split("_")) == 3:
            trash_files.append(i)

In [18]:
len(trash_files)

55

In [ ]:
for file in trash_files:
    os.remove("./embeddings/{}/ready/{}".format(folder,file))
    print("./embeddings/{}/ready/{}\t removed!".format(folder,file))

#building dictionary


In [20]:
# folder = "bert-base-multilingual-cased-liter"

In [21]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [22]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [ ]:
with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
    json.dump(lista_items, f, indent=4)

#Tests

##preparation

In [10]:
# folder = "bert-base-multilingual-cased-liter"

In [11]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [12]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])
def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [13]:
def save_table(df, name):
    df.to_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

def get_table( name):
    return pd.read_csv("./embeddings/{}/tests/{}.csv".format(folder, name),index_col="type")

In [14]:
get_word_m(dct1["мама"], dct1, 10)

{'мама': 1.0,
 'мами': 0.8401787877082825,
 'марина': 0.8497973680496216,
 'мария': 0.8466612100601196,
 'марка': 0.8400037288665771,
 'маска': 0.8512235283851624,
 'масса': 0.8418150544166565,
 'матерей': 0.8463189005851746,
 'матерью': 0.8585536479949951,
 'мать': 0.8620105385780334}

##GATS

###generate test

In [15]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [16]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [17]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [18]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [19]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [20]:
with open(os.path.join("./embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [21]:
with open("tests/gats/ready.pic", "br") as f:
    words_rus = pc.load( f)

In [22]:
def cut_b(text):
    # return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]
    return text

###3cosadd

In [57]:
def cos_add(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                m.append(cut_b(i))
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [58]:
res_df1 = cos_add(dct1)

In [59]:
save_table(res_df1, "3cosadd_res")

In [ ]:
# res_df1[res_df1["group"] == ": gram2-opposite"]

In [60]:
res_df1["group"].unique()

array([': capital-world', ': family', ': gram1-adjective-to-adverb',
       ': gram2-opposite', ': gram3-comparative',
       ': gram6-nationality-adjective'], dtype=object)

In [61]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Capital-countries,0.821504,0.001117,895
Opposite,0.842050,0.033333,30
Adjective adverb,0.866413,0.264069,462
Comparative,0.866738,0.310000,600
Nationality adjective,0.884483,0.406299,635
Family,0.865871,0.500000,20


In [62]:
save_table(res_sort_1, "3cosadd_itog")

In [ ]:
# cut_b("камбоджа")

###3cosavg

In [55]:
def cos_avg(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                    m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                               dct[m[1]].numpy(), 
                                               dct[m[3]].numpy(), 
                                               dct[m[2]].numpy(),
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [41]:
res_df1_avg = cos_avg(dct1)
res_df1_avg.index = res_df1_avg["key"]
add_names(res_df1_avg)
res_df1_avg.index = res_df1_avg["type"]
res_df1_avg[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,рабат,0.923438,рабат,0.923162,0.999091,895
Family,отец,0.945530,отец,0.945530,1.000000,20
Adjective adverb,ощутимый,0.950944,ощутимый,0.950944,1.000000,462
Opposite,приемлемый,0.924010,приемлемый,0.924010,1.000000,30
Comparative,тонкий,0.945294,тонкий,0.945294,1.000000,600
Nationality adjective,германия,0.941876,германия,0.941902,0.999998,635


In [50]:
save_table(res_df1_avg, "3cosavg")

In [56]:
get_table( "3cosavg")

,w1,p1,w2,p2,prob,key,count,type.1
type,,,,,,,,
Capital-countries,рабат,0.923438,рабат,0.923162,0.999091,: capital-world,895,Capital-countries
Family,отец,0.945530,отец,0.945530,1.000000,: family,20,Family
Adjective adverb,ощутимый,0.950944,ощутимый,0.950944,1.000000,: gram1-adjective-to-adverb,462,Adjective adverb
Opposite,приемлемый,0.924010,приемлемый,0.924010,1.000000,: gram2-opposite,30,Opposite
Comparative,тонкий,0.945294,тонкий,0.945294,1.000000,: gram3-comparative,600,Comparative
Nationality adjective,германия,0.941876,германия,0.941902,0.999998,: gram6-nationality-adjective,635,Nationality adjective


###3cosmul


In [24]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [27]:
def cos_mul(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
            # else:
            #     m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word_mul(m[0], m[1], m[3], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            # print(m[2], pred)
    return res_df

In [28]:
for key in words_rus:
    res_df1_mul = cos_mul(key, dct1)
    save_table(res_df1_mul, "3cosmul_res_{}".format(key))
    print("saved 3cosmul_res_{}".format(key))



saved 3cosmul_res_: capital-world
saved 3cosmul_res_: family
saved 3cosmul_res_: gram1-adjective-to-adverb
saved 3cosmul_res_: gram2-opposite
saved 3cosmul_res_: gram3-comparative
saved 3cosmul_res_: gram6-nationality-adjective


In [ ]:

res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]
save_table(res_df_1_mul, "3cosmul_itog")

In [ ]:

save_table(res_df1_mul, "3cosmul_res")
save_table(res_df_1_mul, "3cosmul_itog")

In [ ]:
# res_df1_mul

###mix table

In [ ]:
ddd1 = pd.DataFrame()
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAdd")
ddd1 = ddd1.merge(get_r1(res_df_1_mul[["cosine", "result", "count"]], "3CosMul"), on = "type")
r1 = res_df1_avg[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAvg"), on = "type")
ddd1

,cosine,result,count,cosine,result,count,cosine,count
metric,3CosAdd,3CosAdd,3CosAdd,3CosMul,3CosMul,3CosMul,3CosAvg,3CosAvg
type,,,,,,,,
Comparative,0.882311,0.0,6,0.919653,0.0,6,1,6
Family,0.848026,0.5,2,0.921125,0.5,2,1,2


In [ ]:
ddd1[["result", "cosine", "count"]]

result            cosine                     count                
metric      3CosAdd 3CosMul   3CosAdd   3CosMul 3CosAvg 3CosAdd 3CosMul 3CosAvg
type                                                                           
Comparative     0.0     0.0  0.882311  0.919653       1       6       6       6
Family          0.5     0.5  0.848026  0.921125       1       2       2       2